<a href="https://colab.research.google.com/github/Voland24/ModernComputerVisionPytorch/blob/main/SequentialMethodBuildingNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sequential Method for building a Neural Network**

Instead of building our own class for the NN model, we can use the Sequential class and simplify the process

In [1]:
x = [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[7],[11],[15]]

In [2]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [13]:
class MyDataset(Dataset):
  def __init__(self, x, y):
    self.x = torch.tensor(x).float().to(device)
    self.y = torch.tensor(y).float().to(device)
  def __getitem__(self, index):
    return self.x[index], self.y[index]
  def __len__(self):
    return len(self.x)


In [14]:
ds = MyDataset(x,y)
d1 = DataLoader(ds, batch_size = 2, shuffle = True)


Now comes the model init

In [5]:
model = nn.Sequential(
    nn.Linear(2,8),
    nn.ReLU(),
    nn.Linear(8,1)
).to(device)


Installing packages neccessary to print the summary of the model

In [8]:
from torchsummary import summary

summary(model, (1,2)) #name, input_size

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 8]              24
              ReLU-2                 [-1, 1, 8]               0
            Linear-3                 [-1, 1, 1]               9
Total params: 33
Trainable params: 33
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [17]:
loss_func = nn.MSELoss()
from torch.optim import SGD
opt = SGD(model.parameters(), lr = 0.001)
import time
loss_history = []
start = time.time()
for _ in range(100):
  for ix, iy in d1:
    opt.zero_grad()
    loss_value = loss_func(model(ix), iy)
    loss_value.backward()
    opt.step()
    loss_history.append(loss_value)
end = time.time()
print(end - start)



0.0721426010131836


In [18]:
val = [[8,9],[10,11],[1.5,2.5]]

model(torch.tensor(val).float().to(device))

tensor([[17.1411],
        [21.2419],
        [ 3.8138]], grad_fn=<AddmmBackward0>)

# **Saving and loading a model**

We want to save out model after training and load some model so we don't have to train it but we can use it for inference.

We save the weights and biases of the model in a dictionary, per layer, so we know how to map it back when laoding it.

In [20]:
model.state_dict()

#key/value pairs where the keys are the name of the layer and whether we describe the weights or
#the biases
#and the values are the tensors themselves, values of weights or biases

OrderedDict([('0.weight', tensor([[-0.4386, -0.5338],
                      [-0.3389, -0.6940],
                      [ 0.8834,  0.7201],
                      [ 0.8093, -0.2746],
                      [ 0.7348, -0.2850],
                      [ 0.1053,  0.5454],
                      [ 0.6590,  0.1135],
                      [-0.2307, -0.1246]])),
             ('0.bias',
              tensor([-0.4876, -0.3443, -0.2282,  0.2827,  0.5661, -0.5595, -0.2439,  0.0920])),
             ('2.weight',
              tensor([[ 0.1927, -0.1048,  0.8436,  0.3701,  0.4298,  0.1252,  0.2912,  0.2870]])),
             ('2.bias', tensor([0.2391]))])

**GOOD PRACTICE**

Before saving a model, transfer it from GPU to CPU mem. This will save them as CPU tensors and allow the model to be loaded on a machine that doesn't have PGU capabilities

In [ ]:
torch.save(model.to("cpu").state_dict(), 'path/model.pth')

How to load the model saved by state_dict()?


In [ ]:
state_dict = torch.load('path/model.pth')
model.load_state_dict(state_dict)
model.to(device) #remember,it was on cpu
model(torch.tensor(val).float().to(device)) #use for inference